# Solana Perpetuals Python Client - Interactive Tutorial

This Jupyter notebook provides an interactive way to explore the Solana Perpetuals program using Python.

## Prerequisites

Make sure you have installed the required packages:
```bash
pip install -r requirements.txt
jupyter notebook
```

In [ ]:
# Import required libraries
import asyncio
import sys
import os

# Add the current directory to Python path to import our client
sys.path.append(os.getcwd())

from client import (
    PerpetualsClient, 
    PositionMonitor,
    price_to_program, 
    price_from_program,
    size_to_program,
    size_from_program
)
from solana.keypair import Keypair
from solana.publickey import PublicKey

print("✅ Imports successful!")

## Configuration

Update these values with your actual program deployment:

In [ ]:
# Configuration - Update these values!
RPC_URL = "https://api.devnet.solana.com"
PROGRAM_ID = "YOUR_PROGRAM_ID_HERE"  # Replace with deployed program ID
USER_TOKEN_ACCOUNT = "YOUR_USDC_TOKEN_ACCOUNT"  # Replace with your USDC token account

# Generate a keypair for testing (in practice, load from file)
payer = Keypair()

print(f"🔑 Wallet: {payer.public_key}")
print(f"🌐 RPC: {RPC_URL}")
print(f"📝 Program ID: {PROGRAM_ID}")

## Initialize Client

Create the perpetuals client and test basic functionality:

In [ ]:
# Initialize the client
client = PerpetualsClient(RPC_URL, payer, PROGRAM_ID)

# Test PDA generation
vault_pda, vault_bump = client.get_program_authority()
position_pda, pos_bump = client.get_position_address(payer.public_key)
market_pda, market_bump = client.get_market_state_address()

print(f"🏦 Vault PDA: {vault_pda} (bump: {vault_bump})")
print(f"👤 Position PDA: {position_pda} (bump: {pos_bump})")
print(f"📊 Market PDA: {market_pda} (bump: {market_bump})")

## Price Conversion Utilities

Test the price conversion functions:

In [ ]:
# Test price conversions
prices = [100.0, 100.50, 99.99, 0.01, 1000.0]

print("Price Conversion Tests:")
print("Human Price → Program → Human")
print("-" * 35)

for price in prices:
    program_price = price_to_program(price)
    converted_back = price_from_program(program_price)
    print(f"${price:>8.2f} → {program_price:>12} → ${converted_back:>8.2f}")

# Test size conversions
sizes = [1.0, 0.5, 10.0, 0.001]

print("\nSize Conversion Tests:")
print("Human Size → Program → Human")
print("-" * 32)

for size in sizes:
    program_size = size_to_program(size)
    converted_back = size_from_program(program_size)
    print(f"{size:>8.3f} → {program_size:>12} → {converted_back:>8.3f}")

## Market State Query

Check the current market state (if program is deployed):

In [ ]:
# Query market state
async def check_market_state():
    try:
        market_state = await client.get_market_state()
        
        if market_state:
            print("📊 Market State:")
            print(f"   Funding Index: {market_state.funding_index:,}")
            print(f"   Funding Rate/Slot: {market_state.funding_rate_per_slot:,}")
            print(f"   Open Interest: {size_from_program(market_state.open_interest):.4f}")
            print(f"   Mark Price: ${price_from_program(market_state.mark_price):.2f}")
            print(f"   Last Funding Slot: {market_state.last_funding_slot:,}")
            print(f"   PDA Bump: {market_state.bump}")
        else:
            print("❌ Market state not found - program may not be deployed or initialized")
            
    except Exception as e:
        print(f"❌ Error querying market state: {e}")

# Run the async function
await check_market_state()

## Position Query

Check if we have any existing positions:

In [ ]:
# Query position
async def check_position():
    try:
        position = await client.get_position(payer.public_key)
        
        if position:
            print("👤 Current Position:")
            print(f"   Owner: {position.owner}")
            print(f"   Size: {size_from_program(position.base_amount):.4f}")
            print(f"   Collateral: ${price_from_program(position.collateral):.2f}")
            print(f"   Entry Price: ${price_from_program(position.entry_price):.2f}")
            print(f"   Last Funding Index: {position.last_funding_index}")
            
            # Determine position type
            if position.base_amount > 0:
                print(f"   Type: 🟢 Long")
            elif position.base_amount < 0:
                print(f"   Type: 🔴 Short")
            else:
                print(f"   Type: ⚪ No Position")
        else:
            print("👤 No position found for this wallet")
            
    except Exception as e:
        print(f"❌ Error querying position: {e}")

# Run the async function
await check_position()

## Simulation: Position Health Calculation

Let's simulate position health calculations without actually opening a position:

In [ ]:
# Simulate position scenarios
from client import Position

# Create mock positions for testing
scenarios = [
    {
        "name": "Healthy Long Position",
        "base_amount": size_to_program(1.0),     # 1 unit long
        "collateral": price_to_program(200),     # $200 collateral
        "entry_price": price_to_program(100),    # $100 entry
        "current_price": price_to_program(105),  # $105 current
    },
    {
        "name": "Risky Long Position",
        "base_amount": size_to_program(1.0),     # 1 unit long
        "collateral": price_to_program(140),     # $140 collateral
        "entry_price": price_to_program(100),    # $100 entry
        "current_price": price_to_program(95),   # $95 current (loss)
    },
    {
        "name": "Liquidatable Position",
        "base_amount": size_to_program(1.0),     # 1 unit long
        "collateral": price_to_program(120),     # $120 collateral
        "entry_price": price_to_program(100),    # $100 entry
        "current_price": price_to_program(85),   # $85 current (big loss)
    }
]

print("Position Health Analysis:")
print("=" * 60)

for scenario in scenarios:
    # Create mock position
    position = Position(
        owner=payer.public_key,
        base_amount=scenario["base_amount"],
        collateral=scenario["collateral"],
        last_funding_index=0,
        entry_price=scenario["entry_price"]
    )
    
    # Calculate metrics
    current_price = scenario["current_price"]
    
    # Health calculation
    health = await client.calculate_position_health(position, current_price)
    
    # PnL calculation
    pnl = await client.calculate_unrealized_pnl(position, current_price)
    
    # Display results
    print(f"\n📈 {scenario['name']}:")
    print(f"   Entry Price: ${price_from_program(scenario['entry_price']):.2f}")
    print(f"   Current Price: ${price_from_program(current_price):.2f}")
    print(f"   Collateral: ${price_from_program(scenario['collateral']):.2f}")
    print(f"   Health Ratio: {health:.3f} {'✅' if health >= 1.5 else '⚠️' if health >= 1.2 else '🚨'}")
    print(f"   Unrealized PnL: ${price_from_program(pnl):.2f} {'📈' if pnl > 0 else '📉' if pnl < 0 else '➡️'}")
    print(f"   Liquidatable: {'🚨 YES' if health < 1.5 else '✅ No'}")

## Position Monitor Demo

Test the position monitoring functionality:

In [ ]:
# Test position monitor
monitor = PositionMonitor(client)

async def demo_position_summary():
    print("📋 Position Summary Demo:")
    print("-" * 30)
    
    # Get summary for our wallet (even if no position exists)
    summary = await monitor.get_position_summary(payer.public_key)
    
    if summary["exists"]:
        if "error" in summary:
            print(f"❌ Error: {summary['error']}")
        else:
            print(f"Owner: {summary['owner']}")
            print(f"Position Type: {summary['position_type']}")
            print(f"Size: {summary['size']:.4f}")
            print(f"Collateral: ${summary['collateral']:.2f}")
            print(f"Entry Price: ${summary['entry_price']:.2f}")
            print(f"Mark Price: ${summary['mark_price']:.2f}")
            print(f"Health Ratio: {summary['health_ratio']:.3f}")
            print(f"Unrealized PnL: ${summary['unrealized_pnl']:.2f}")
            print(f"Liquidatable: {'🚨 YES' if summary['is_liquidatable'] else '✅ No'}")
    else:
        print("👤 No position exists for this wallet")

await demo_position_summary()

## Transaction Simulation

Show what the transaction data would look like (without actually sending):

In [ ]:
# Simulate transaction data creation
import struct

def simulate_open_position_instruction():
    # Example parameters
    base_delta = size_to_program(1.0)        # 1 unit long
    collateral_delta = price_to_program(150) # 150 USDC
    entry_price = price_to_program(100.50)   # $100.50
    
    # Create instruction data
    instruction_data = bytearray(25)
    instruction_data[0] = 0  # INSTRUCTION_OPEN_POSITION
    instruction_data[1:9] = struct.pack('<q', base_delta)
    instruction_data[9:17] = struct.pack('<Q', collateral_delta)
    instruction_data[17:25] = struct.pack('<Q', entry_price)
    
    print("📝 Open Position Instruction Data:")
    print(f"   Instruction Tag: {instruction_data[0]}")
    print(f"   Base Delta: {size_from_program(base_delta):.4f} units")
    print(f"   Collateral Delta: ${price_from_program(collateral_delta):.2f}")
    print(f"   Entry Price: ${price_from_program(entry_price):.2f}")
    print(f"   Raw Data (hex): {instruction_data.hex()}")
    print(f"   Data Length: {len(instruction_data)} bytes")
    
    return instruction_data

simulate_open_position_instruction()

## Cleanup

Always close the client connection when done:

In [ ]:
# Clean up
await client.close()
print("✅ Client connection closed")

## Next Steps

To use this client with a real deployed program:

1. **Deploy the Program**: Use the build and deploy scripts in the main directory
2. **Update Configuration**: Replace `PROGRAM_ID` with your deployed program ID
3. **Create Token Accounts**: Set up USDC token accounts for collateral
4. **Fund Accounts**: Get devnet SOL and USDC for testing
5. **Test Operations**: Try opening positions, updating funding, etc.

### Useful Commands:

```bash
# Deploy to devnet
cd ..
scripts\build.bat
scripts\deploy.bat devnet

# Create USDC token account
spl-token create-account EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v

# Get devnet SOL
solana airdrop 2
```

### Safety Reminders:

- ⚠️ This is educational code - not production ready
- 🧪 Always test on devnet first
- 🔐 Never share private keys
- 📚 Study the Rust program code to understand the mechanics
- 🛡️ Implement proper error handling for production use